In [95]:
import pandas as pd
import numpy as np
import json

pd.options.display.width=10000
pd.options.display.max_columns=1000


In [ ]:
df = pd.read_json('data.json', lines=True)
display(df.head())
df.shape

In [107]:
users = pd.DataFrame.from_records(df["user"].values)
users.head()

,contributors_enabled,created_at,default_profile,default_profile_image,description,favourites_count,follow_request_sent,followers_count,following,friends_count,geo_enabled,id,id_str,is_translator,lang,listed_count,location,name,notifications,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_banner_url,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,protected,screen_name,statuses_count,time_zone,translator_type,url,utc_offset,verified
0,False,Wed May 16 15:01:53 +0000 2012,False,False,"ロリコンで虹専の救いようのない21歳です！現在留学でKansas,wichita在住。最近サ...",23071,None,467,None,439,True,581906827,581906827,False,ja,14,"Kansas, USA",Nazell@アメリカ留学 （夏休み）,None,EDECE9,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,True,https://pbs.twimg.com/profile_banners/58190682...,http://pbs.twimg.com/profile_images/9199536738...,https://pbs.twimg.com/profile_images/919953673...,088253,FFFFFF,DDEEF6,333333,True,False,nazeru2,20547,None,none,https://inara.cz/cmdr/103080/,None,False
1,False,Sun Jul 07 18:45:00 +0000 2013,True,False,None,946,None,63,None,434,True,1575803474,1575803474,False,en,0,None,Galahad,None,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/15758034...,http://pbs.twimg.com/profile_images/9920618850...,https://pbs.twimg.com/profile_images/992061885...,1DA1F2,C0DEED,DDEEF6,333333,True,False,CONGOBILL1,1724,None,none,None,None,False
2,False,Sun Aug 14 14:46:09 +0000 2011,False,False,Curious lead software & web #developer #Java #...,190,None,111,None,192,False,354923441,354923441,False,fr,6,Rennes,Alban Berthout 💡,None,000000,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/35492344...,http://pbs.twimg.com/profile_images/9910234076...,https://pbs.twimg.com/profile_images/991023407...,3B94D9,000000,000000,000000,False,False,AlbanBerthout,355,None,none,None,None,False
3,False,Sat Jul 05 08:56:50 +0000 2008,False,False,Passionné de technologie. #CTO/#COO/#coachagil...,7084,None,14770,None,7102,True,15324359,15324359,False,fr,554,"Paris, France",pmedina,None,000000,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/15324359...,http://pbs.twimg.com/profile_images/9753511040...,https://pbs.twimg.com/profile_images/975351104...,ABB8C2,000000,000000,000000,False,False,pmedina,68095,None,none,http://fr.linkedin.com/in/pierremedina,None,False
4,False,Sat May 12 09:09:39 +0000 2012,False,False,Little Monsłer /《La historia os condenará y no...,158613,None,1099,None,899,True,577889339,577889339,False,es,27,Mataró,LG6 is coming,None,000000,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/57788933...,http://pbs.twimg.com/profile_images/9917322170...,https://pbs.twimg.com/profile_images/991732217...,3A02B2,000000,DDEEF6,333333,True,False,alexcorts22,98597,None,regular,http://ladygaga.com,None,False


In [108]:
retweets = df["retweeted_status"].apply(pd.Series)
retweets.head()

/Users/devashishsood/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:43: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
/Users/devashishsood/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)
/Users/devashishsood/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:43: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
/Users/devashishsood/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,place,possibly_sensitive,quote_count,reply_count,retweet_count,retweeted,source,text,truncated,user,0,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,scopes
0,NaN,None,Sun May 27 15:37:36 +0000 2018,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,False,low,None,1.000763e+18,1000762953665138688,None,NaN,None,NaN,None,False,ja,None,NaN,0.0,0.0,1.0,False,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Python中型船で一番使いやすいらしいから長く乗ることになりそう,False,"{'id': 3101551238, 'id_str': '3101551238', 'na...",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,None,Mon May 28 07:24:13 +0000 2018,"[0, 19]","{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 1001001180003225600, 'id_str...",NaN,2.0,False,low,None,1.001001e+18,1001001181156651008,None,NaN,None,NaN,None,False,en,None,False,0.0,0.0,1.0,False,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",I'm learning Python https://t.co/AVxMV3rWZt,False,"{'id': 837070053401112576, 'id_str': '83707005...",NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,None,Sat May 26 10:30:27 +0000 2018,NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,NaN,6.0,False,low,None,1.000323e+18,1000323270296702976,fchantrel,9.868863e+17,986886337608503296,329009436.0,329009436,False,fr,None,False,0.0,0.0,4.0,False,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Et si vous êtes allergique au python et que vo...,False,"{'id': 329009436, 'id_str': '329009436', 'name...",NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,None,Mon May 28 03:28:03 +0000 2018,"[0, 140]","{'hashtags': [{'text': 'BigData', 'indices': [...",NaN,{'full_text': 'Choosing a Machine Learning Fra...,15.0,False,low,None,1.000942e+18,1000941744169668610,None,NaN,None,NaN,None,False,en,None,False,0.0,0.0,19.0,False,"<a href=""http://bufferapp.com"" rel=""nofollow"">...",Choosing a Machine Learning Framework. #BigDat...,True,"{'id': 4263007693, 'id_str': '4263007693', 'na...",NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,None,Mon May 28 07:24:13 +0000 2018,"[0, 19]","{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 1001001180003225600, 'id_str...",NaN,5.0,False,low,None,1.001001e+18,1001001181156651008,None,NaN,None,NaN,None,False,en,None,False,0.0,0.0,2.0,False,"<a href=""https://ifttt.com"" rel=""nofollow"">IFT...",I'm learning Python https://t.co/AVxMV3rWZt,False,"{'id': 837070053401112576, 'id_str': '83707005...",NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
retweet_users= retweets["user"].apply(pd.Series)
retweet_users.head()

/Users/devashishsood/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:43: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  union = _union_indexes(indexes)
/Users/devashishsood/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


,contributors_enabled,created_at,default_profile,default_profile_image,description,favourites_count,follow_request_sent,followers_count,following,friends_count,geo_enabled,id,id_str,is_translator,lang,listed_count,location,name,notifications,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_banner_url,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,protected,screen_name,statuses_count,time_zone,translator_type,url,utc_offset,verified,0
0,False,Sat Mar 21 12:24:45 +0000 2015,False,False,( ˘⊖˘)｡oO(いっぱいキメて♪いっぱいシャブれる♪),1237.0,NaN,45.0,NaN,53.0,False,3.101551e+09,3101551238,False,ja,1.0,ship2 ウル,R053🐈Sp@ceCatCMDR🌍,NaN,000000,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/31015512...,http://pbs.twimg.com/profile_images/8743403649...,https://pbs.twimg.com/profile_images/874340364...,F58EA8,000000,000000,000000,False,False,akchn69,9393.0,NaN,none,None,NaN,False,NaN
1,False,Wed Mar 01 22:40:27 +0000 2017,False,False,"a light-hearted, 'self-aware' socialist satire...",7.0,NaN,22360.0,NaN,4.0,False,8.370701e+17,837070053401112576,False,en,140.0,The Motherland,FULL COMMUNISM,NaN,000000,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/83707005...,http://pbs.twimg.com/profile_images/8370745339...,https://pbs.twimg.com/profile_images/837074533...,E81C4F,000000,000000,000000,False,False,LaterCapitalism,7392.0,NaN,none,None,NaN,False,NaN
2,False,Mon Jul 04 12:00:19 +0000 2011,True,False,Software Architect\r\nInterested in open sourc...,179.0,NaN,169.0,NaN,170.0,False,3.290094e+08,329009436,False,fr,31.0,"Rennes, France",fchantrel,NaN,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,NaN,http://pbs.twimg.com/profile_images/1453908178...,https://pbs.twimg.com/profile_images/145390817...,1DA1F2,C0DEED,DDEEF6,333333,True,False,fchantrel,1288.0,NaN,none,None,NaN,False,NaN
3,False,Tue Nov 17 02:19:09 +0000 2015,False,False,Ganapathi Pulipaka | CEO | Chief Data Scientis...,113.0,NaN,41380.0,NaN,22924.0,True,4.263008e+09,4263007693,False,en,2944.0,"Los Angeles, CA",Dr. GP Pulipaka,NaN,000000,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/42630076...,http://pbs.twimg.com/profile_images/9983012584...,https://pbs.twimg.com/profile_images/998301258...,DD2E44,000000,000000,000000,False,False,gp_pulipaka,39255.0,NaN,none,https://www.linkedin.com/in/dr-ganapathi-pulip...,NaN,False,NaN
4,False,Wed Mar 01 22:40:27 +0000 2017,False,False,"a light-hearted, 'self-aware' socialist satire...",7.0,NaN,22360.0,NaN,4.0,False,8.370701e+17,837070053401112576,False,en,140.0,The Motherland,FULL COMMUNISM,NaN,000000,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/83707005...,http://pbs.twimg.com/profile_images/8370745339...,https://pbs.twimg.com/profile_images/837074533...,E81C4F,000000,000000,000000,False,False,LaterCapitalism,7392.0,NaN,none,None,NaN,False,NaN


In [110]:
entities = pd.DataFrame.from_records(df["entities"].values)
entities.head()

,hashtags,media,symbols,urls,user_mentions
0,[],NaN,[],[],"[{'screen_name': 'akchn69', 'name': 'R053🐈Sp@c..."
1,[],"[{'id': 1001001180003225600, 'id_str': '100100...",[],[],"[{'screen_name': 'LaterCapitalism', 'name': 'F..."
2,[],NaN,[],[],"[{'screen_name': 'fchantrel', 'name': 'fchantr..."
3,"[{'text': 'BigData', 'indices': [56, 64]}, {'t...",NaN,[],[],"[{'screen_name': 'gp_pulipaka', 'name': 'Dr. G..."
4,[],"[{'id': 1001001180003225600, 'id_str': '100100...",[],[],"[{'screen_name': 'LaterCapitalism', 'name': 'F..."


In [111]:
display(df.columns)
display(users.columns)
display(entities.columns)
display(retweets.columns)
display(retweet_users.columns)

Index(['contributors', 'coordinates', 'created_at', 'display_text_range', 'entities', 'extended_entities', 'extended_tweet', 'favorite_count', 'favorited', 'filter_level', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place', 'possibly_sensitive', 'quote_count', 'quoted_status', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink', 'reply_count', 'retweet_count', 'retweeted', 'retweeted_status', 'source', 'text', 'timestamp_ms', 'truncated', 'user'], dtype='object')

Index(['contributors_enabled', 'created_at', 'default_profile', 'default_profile_image', 'description', 'favourites_count', 'follow_request_sent', 'followers_count', 'following', 'friends_count', 'geo_enabled', 'id', 'id_str', 'is_translator', 'lang', 'listed_count', 'location', 'name', 'notifications', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_background_tile', 'profile_banner_url', 'profile_image_url', 'profile_image_url_https', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 'protected', 'screen_name', 'statuses_count', 'time_zone', 'translator_type', 'url', 'utc_offset', 'verified'], dtype='object')

Index(['hashtags', 'media', 'symbols', 'urls', 'user_mentions'], dtype='object')

Index(['contributors', 'coordinates', 'created_at', 'display_text_range', 'entities', 'extended_entities', 'extended_tweet', 'favorite_count', 'favorited', 'filter_level', 'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place', 'possibly_sensitive', 'quote_count', 'reply_count', 'retweet_count', 'retweeted', 'source', 'text', 'truncated', 'user', 0, 'quoted_status', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink', 'scopes'], dtype='object')

Index(['contributors_enabled', 'created_at', 'default_profile', 'default_profile_image', 'description', 'favourites_count', 'follow_request_sent', 'followers_count', 'following', 'friends_count', 'geo_enabled', 'id', 'id_str', 'is_translator', 'lang', 'listed_count', 'location', 'name', 'notifications', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_background_tile', 'profile_banner_url', 'profile_image_url', 'profile_image_url_https', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 'protected', 'screen_name', 'statuses_count', 'time_zone', 'translator_type', 'url', 'utc_offset', 'verified', 0], dtype='object')

In [112]:
display(df.shape)
display(users.shape)
display(entities.shape)
display(retweets.shape)
display(retweet_users.shape)

(29181, 36)

(29181, 39)

(29181, 5)

(29181, 36)

(29181, 40)